In [1]:
conversation = """
说话人1 :开始吧，开始吧，我是说话人一我叫王满红，今年28岁。
说话人2 :我是说话人二我是孙玉涛。
说话人3 :我是三号，我是三号。关于一号的发言，我表示质疑。
说话人4 :我是说话人四号，我是演员四号，我非常赞同一号的说话呀。
说话人1 :桌上为什么这么多吃的还有小面包从哪里搞的？
说话人2 :哦，这个都已经注册上了。
说话人4 :我们看一下没有昨天晚上我没注册上我的四号没注册。
说话人4 :上四号刚刚注册。
说话人4 :哪里呀？我只是显示了这个，但是说话人还没还没有显示四号，你看发言人只有三。
说话人4 :还没蹦完。
说话人4 :还没碰完。
说话人4 :你就是那就，那就。那就可以自由聊会儿聊。
说话人2 :但是其实真实的开会的场景，呃，这种重叠讲话的人多吗？其实感觉。
说话人1 :多，他就是一个人还没讲完另外一个人就接着上了。
说话人2 :是的，那在这种情况好像大家表现都。
说话人4 :还有这这些情况下不会让你前面有一个人来讲一句啊，对吧？人有贼多，我靠你那万一两个地域不是相同的，他们前面坐那个做声纹他们，他们就提出这个问题就是如果低于相同的两个人去做声纹，就
是它那个，我觉得80%的那个唤醒错误率了，我都不。
说话人2 :敢，你的意思就是同一个地域的。
说话人4 :他跟一些，那个，那个。
说话人4 :就像龙性别的他那个真的是一模一样的那个子孙也贼哦，不是那个幻影错误率贼高。他说，虽然他搞声纹注册了，但是他们当时承诺的指标是当时上线的指标是就是就是30%嘛，就是唤醒这个误闯率30%
，所以这边还是挺。
说话人4 :高的了。
说话人4 :就说说明他辨识度不高是吧？
说话人4 :呃，对双方辩手都不是很高，就是因为他说那些，嗯，他也说了，华为那些苹果的话都是我们都不行，都不敢，他们好像自己都。
说话人4 :测过的都是都是都不。
说话人4 :守住了，不错。
说话人4 :双方现在最大的问题好像前面。
说话人1 :就是一些鲁网行动。其实如果在他们使用的过程中，就是刚开始的时候提示就是一开始多注册一点长的，我操这***救护车吗？不太好干哦。
说话人1 :是你是你是四号是吧？对啊，这个一直刚才那个厂出来都是在三号。
说话人1 :下面三号才三号。
说话人4 :我不是四号三号。
说话人1 :四号都很模糊。
说话人1 :我是几号？我是几号？
说话人1 :我生日几号？
说话人4 :现在我一直在说话你现在可以分出来这个人是谁吗？
说话人1 :叶山浩，他也是三号。
说话人4 :是吗？我也是三号嘛我是几号？
说话人4 :最终就被你合并了，嗯？
说话人4 :不是不是，不是，是不是？
说话人4 :找找个女生来看一下女生和男生之间。
说话人4 :会不会和平？我靠。
说话人1 :女生区分度高一。
说话人4 :点，女生就会高一点。嗯，异性之间可能区分度高一点。
说话人4 :同性之间，可能这个有点。
说话人3 :现在还有四号吗？一直都是三号的声音，现在是两个人分不出来的。
说话人4 :她也估计也就是类似于生活比较相似度，它结果把三号四号的生物给他提取在一起了，他觉得跟三号也挺近。
说话人4 :的跟四号也挺近，可能三号四号从那个距离图上他。
说话人4 :就靠得很。
说话人2 :近，所以说句后面他就混沌了。
说话人2 :估计是了。
说话人4 :后面后面反正他后面你比较好也是比较上蜘蛛。
说话人4 :他觉得就是明星就行。
说话人1 :他已经初六了，他已经出六了。
说话人2 :运费那就这样子呗，我们到到时候对的逻辑就是说这个时时呢就是容易容易不太准对吧？那就到时候这样子，我明天到时候明天这个逻辑跟她讲，第一个就这个指标，你看今天来不来得及整袋来得及拯
救来得及整来不及整，到时候给他们看一下我们我们那个就是说那个实验室里想的，你就说大家都相当，你不要说谁好。
说话人2 :奇怪，也就说大家其实都差不多，但实际的你就看牛似的，你说这个讯飞的也也这个样子。然后我们也下个结论是比他好还是比他差，还是他有什么优点，他有什么缺点，就你那边多多写一些对吧，然
后那个东西那个那个那个那个，因为你说这个时代这个指标。
说话人2 :统计起来比较比较比较比较难统计哈，都是不同，但是从那个从从。
说话人2 :不管的感受来看是谁的更好对吧。然后再把那个卡片还是做了一个那个离线的吗？那离线的是不是这个结果可以拿过去一回一回放就是拿过去一一上传不就好了嘛，对不对？就拿这个手机上的对不对，
这样子一对比打不进去啊。讯飞讯飞听见讯飞听见打不进去音频那倒不进去，因为你就现在哪个手机开着的。
说话人2 :那不就好了吗？那你你必须要重弄一遍话，大哥，我**咋想的嘞？我操。
说话人2 :红歌，所以就是说这个逻辑这个。
说话人4 :这个也没导出来，应该是个完整的页面。
说话人2 :不应该是每个人的样子，就让它有个预期就做完了。这个东西虽然现在是不是特别稳定，但是现在业界就是这个样子，但我们态度上我们是很好，我们肯定会配合你去优化那些东西，哪些问题我们也总
结出来，对吧？就就就就这个样子啊，嗯，就行了呀。
说话人2 :嗯，好。
说话人2 :你。你明白了吗？就这个。
说话人2 :这个逻辑是可以的。
说话人2 :嗯，应该可以到吧，你可以随便。
说话人2 :就互操。
说话人1 :可以的，可以的，我们测一下短剧的效果。
说话人3 :那个合并的逻辑是。
说话人3 :怎么搞你，你也来一句。
说话人5 :何必那么过去就就是。
说话人4 :如果我们俩人说话非常相近的话他不是就两个人你说话马上说的话他会合并把它。
说话人4 :拽出来成一个人，对吗？
说话人1 :有没有好吃的呀？
说话人1 :这都是些什么零食啊，都是一些垃圾零食。
说话人2 :我还是二号嘛，我票都没有。
说话人2 :啊，你不要跑过来。
说话人2 :讲没必要没事儿这***不正常吗？
说话人1 :开个会？对啊正常一一米多的距离吗？
说话人3 :我到现在不知道我是三号还是四号，你给一个准确的结论吧。
说话人3 :我变成变成五号了。
说话人1 :因为你在吃肉。
说话人1 :吃东西讯飞给的你是六。
说话人3 :嗯，现在东西吃完了啊能回归到四号身份了吗？还是。
说话人4 :应该应该回归不了了，我觉得。
说话人3 :嗯四号四号了嗯？
说话人3 :哎那。
说话人2 :比如说像这样，我们收敛一点就是这个东西能能能能支撑刚刚我说的那个事情，就比如有一个直观的这个那个主观的那个那个比较，所以会好一点，或者一个主观的指标弄出来，或者就结结个结节一段
时间，呃，今天下午让他们陪你去标一下，还是怎么说可以啊。
说话人2 :约十分钟，约十分钟。嗯，能得到这个结论，其实就是说如果能得到结论是这个跟这个是相当的，然后这个是比那个经验里离线的倒进去的要差一些，基本上就就能说得通了吗？逻辑上就可以对的上可
以可以。
说话人6 :是吧？这个感觉是不是没有之前那个靠谱了，最近感觉效果变差了，感觉也可能是不稳定。
说话人3 :幸亏这个还会更新吗？他那个联网。
说话人4 :就你们这个应该更新会。
说话人1 :更新吧，那谁知道呢？他更不更新，我们也不知道。
说话人1 :好，我吃东西的时候他也知道我是。
说话人1 :一级还可以，其实。
说话人2 :对吧？那我们就在群里说一下这个东西已经更新了，让他们在体验体验，就是说明天再对一下，你把这个问题再整整咱这个东西说实话，妈的能做的也就那个样子，那个东西我再回去把那个发布会的讯
飞的那个东西再研究研究。因为昨天王总就问了，你把这个结果等等，就按照那个思路，就把那些问题再以以他们不太懂。
说话人2 :这个专业能听得懂的维度，来来来，讲一讲懂的意思了吗？好吧，嗯，好吧，好吧，你有哪些问题，然后多弄一些，就表明我们这个东西还是经常用，用了很多的那个东西太少了。好吧，嗯。
说话人1 :你刚才那段话我需要。
说话人4 :剪掉是感冒了吗？我。
说话人2 :感觉是感冒了呀。
说话人2 :我**这二七了呀。
说话人2 :带冰袋冰袋。
说话人3 :上回我昨晚还在战斗呢。
说话人4 :你错了还跟我说你回家有事儿结果回家呆着。
说话人6 :真他妈是已经是那都11点了11点了，大哥。
说话人4 :**你这有事儿。
说话人3 :11点干到。
说话人3 :凌晨12点半，一点半要12点半。
说话人2 :加入你这一步，这一步打了。
说话人3 :都干干干干不动了。
说话人3 :昨天我们不也在。
说话人3 :打一会儿回来一看，你们已经开了。
说话人4 :你们晚上几个哪几个？
说话人2 :可热闹了，前天下午要不把你拉。
说话人4 :进去，哇，那个手游我是一打出汗。
说话人2 :我江苏给他介绍一下我们的盛况好吧，明天晚上的。
说话人3 :夏博士刀塔战神。
说话人4 :我怎么都不打Dota我打断我打断由我不。
说话人4 :打手游手游。
说话人4 :说什么出汗？
说话人7 :不行，消费者对1607这边待会要开交流会四点。
说话人2 :四点是吧，我看现在几点好，我们马上就结束了想为什么这么多吃的吃的。
说话人2 :糖豆，你也要参加。
说话人2 :我要参加。
说话人2 :啥？我说啥会。
说话人7 :呀，应届生交流。
说话人2 :我操这还要交流啥喂？
说话人1 :好好好好把垃圾整理一下吧，兄弟们。
说话人4 :哦，怪不得我。
说话人2 :待会可以拿过来让他们测一下你把这个先倒过去四点的时候。
说话人2 :就拿过来。
说话人2 :测，看看交流会我们这个情况，因为。
说话人2 :好多人，那大方没关系我们就极限测试啊。
说话人1 :那一会你拿来呗啊。
说话人2 :我拿过来我就在这儿好不好那你不是要。
说话人2 :参加吗？我不是说没人。
说话人2 :通知我要参加，我不知道没人通知我。
说话人2 :不是发邮件了吗？
说话人2 :9点51好的，你先把它拿过去抖一下，让标注原先去标，然后我们。
说话人2 :继续再弄吧。
说话人2 :OKOK那那个。那个讯飞听见。
"""

gpt_summary = """
会议主题：声纹识别系统优化讨论会

与会人：

王满红（说话人一）
孙玉涛（说话人二）
三号（未确认身份）
演员四号（未确认身份）
叶山浩（未确认身份）
会议议程：

讨论声纹识别系统的性能问题，特别是重叠讲话时的效果。
分析声纹识别系统在同一地域和不同地域之间的辨识度。
探讨声纹识别系统对于男性和女性之间的区分度。
讨论声纹识别系统的更新频率和稳定性。
会议结论：

声纹识别系统在重叠讲话时效果不佳，需要优化处理。
同一地域的声纹识别区分度较低，需要进一步提高。
异性之间的声纹识别区分度较高，同性之间可能存在问题。
更新声纹识别系统是必要的，但需注意稳定性和更新频率。
下一步行动：

对声纹识别系统进行优化，特别是在重叠讲话场景下的处理。
进一步研究提高同一地域的声纹识别区分度。
继续探索异性和同性之间的声纹识别区分度，并进行数据收集与分析。
优化更新机制，确保声纹识别系统的稳定性和性能。
备注：会议中出现了一些未确认身份的说话人，需要进一步确认和记录。
"""

In [1]:
from datasets import load_dataset
import json
import requests
from collections import defaultdict

from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from inference_api import YiyanInferenceApi, HfInferenceApi
import regex

from langchain.llms import Tongyi
from wenxinyiyan import WxYiyan

def kwargs_test(**kwargs) :
    for key, val in kwargs.items() :
        print(f"{key}:{val}")
    print(kwargs['prompt'])

kwargs_test(prompt='Hello', temperature=0.2, top_p=0.9)

llm = WxYiyan(model_name = 'ERNIE-Bot')
llm('tell me a joke!')


/Users/wujianmin/miniforge3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


prompt:Hello
temperature:0.2
top_p:0.9
Hello
Yiyan api key: 6Rs1EueWFULgAnPUS2HQDx2H, Yiyan sec key: Fl3R1XOeBdQfIcHng4Ah25RHzuSQxChT
{"messages": [{"role": "user", "content": "tell me a joke!"}], "top_p": 0.8, "temperature": 0.2, "penalty_score": 1.0}
{'id': 'as-sde4epcz32', 'object': 'chat.completion', 'created': 1691736393, 'result': '当然，这里有一个小笑话：\n\n一只乌龟和一只兔子在森林里比赛跑步。\n\n乌龟说：“你知道吗，兔子？我曾经从太平洋一直走到大西洋。”\n\n兔子回答：“那还不容易，我也会。”\n\n乌龟惊讶地说：“真的吗？那你走的是哪条路？”\n\n兔子回答：“当然是从太平洋银行到亚特兰大。”', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 5, 'completion_tokens': 97, 'total_tokens': 102}}


'当然，这里有一个小笑话：\n\n一只乌龟和一只兔子在森林里比赛跑步。\n\n乌龟说：“你知道吗，兔子？我曾经从太平洋一直走到大西洋。”\n\n兔子回答：“那还不容易，我也会。”\n\n乌龟惊讶地说：“真的吗？那你走的是哪条路？”\n\n兔子回答：“当然是从太平洋银行到亚特兰大。”'

In [3]:
eb_llm = YiyanInferenceApi("ERNIE-Bot", system="你是OPPO助手，请热情准确地回答用户的问题。", debug=True)
print(eb_llm("Hello!",  temperature = 0.1, top_p = 0.9, penalty_score = 1.0))

eb_turbo_llm = YiyanInferenceApi("ERNIE-Bot-turbo", system="你是OPPO助手，请热情准确地回答用户的问题。", debug=True)
print(eb_turbo_llm("who are you?",  temperature = 0.1, top_p = 0.9, penalty_score = 1.0))

ValueError: "YiyanInferenceApi" object has no field "repo_id"

In [2]:
from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from wenxinyiyan import WxYiyan

llm = WxYiyan(mod"ERNIE-Bot", debug=False)

# summerization template
format_template = """
你是一位认真仔细的会议助手，根据以下会议讨论中不同用户发言的内容，请重点总结提取对话核心内容，包括会议主题、会议议程、会议结论以及下一步行动。对话内容如下：\n
{conversation} \n\n
总结的核心内容如下：\n
"""

dialogs = conversation.split("\n")

conversations = []
conversation_chunk = []

chunk_len = 0
CONTEXT_LEN = 1800

for diag in dialogs :
    diag = ':'.join(diag.split(':')[1:])
    chunk_len += len(diag)
    if chunk_len < CONTEXT_LEN :
        conversation_chunk.append(diag)
    else :
        conversations.append("\n".join(conversation_chunk))
        conversation_chunk = []
        conversation_chunk.append(diag)
        chunk_len = len(diag)

if len(conversation_chunk) > 0: 
    conversations.append("\n".join(conversation_chunk))

print(f"len_dialog={len(dialogs)}, len_chunks={len(conversations)}")
# print(f"{len(conversations[0])}, {len(conversations[1])}")

summerizations = []

for (i, c) in enumerate(conversations) :
    example_prompt = PromptTemplate.from_template(format_template)
    grounded_prompt = example_prompt.format(conversation = c)
    completion = llm(grounded_prompt)
    summerizations.append(f"{completion}")
    print(f"###部分纪要-{i}")
    print(f"{completion}")

# overall summerization template
format_template = """
你是一位认真仔细的会议助手，请根据如下几个分段的会议核心内容，总结一份完整的会议纪要，包括会议主题、会议议程、会议结论以及下一步行动。
要求会议议程不要超过10条，会议结论用一句话简单概括。分段会议核心内容如下：\n\n
{summerizations} \n\n
最终总结的会议纪要的如下：\n\n
"""

example_prompt = PromptTemplate.from_template(format_template)
grounded_prompt = example_prompt.format(summerizations = "\n\n".join(summerizations))
# print(grounded_prompt)
completion = llm(grounded_prompt)

print(f"###汇总纪要")
print(f"{completion}")



Yiyan api key: 6Rs1EueWFULgAnPUS2HQDx2H, Yiyan sec key: Fl3R1XOeBdQfIcHng4Ah25RHzuSQxChT


NameError: name 'conversation' is not defined

In [2]:
import langchain
from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain, StuffDocumentsChain, ReduceDocumentsChain
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.schema.document import Document
from wenxinyiyan import WxYiyan

# llm = YiyanInferenceApi("ERNIE-Bot", debug=False)
llm = WxYiyan(temperature=0.1, model_name = "ERNIE-Bot")
# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")

dialogs = conversation.split("\n")

conversations = []
conversation_chunk = []

chunk_len = 0
CONTEXT_LEN = 1800

for diag in dialogs :
    diag = ':'.join(diag.split(':')[1:])
    chunk_len += len(diag)
    if chunk_len < CONTEXT_LEN :
        conversation_chunk.append(diag)
    else :
        conversations.append(Document(page_content="\n".join(conversation_chunk)))
        conversation_chunk = []
        conversation_chunk.append(diag)
        chunk_len = len(diag)

if len(conversation_chunk) > 0: 
    conversations.append(Document(page_content="\n".join(conversation_chunk)))

print(f"len_dialog={len(dialogs)}, len_chunks={len(conversations)}")
# print(f"{len(conversations[0])}, {len(conversations[1])}")

langchain.debug = False
# summerization template
format_template = """
请根据以下会议讨论中不同用户发言，总结提取对话的核心内容。对话内容如下在 >>> 和 <<< 之间：\n
>>> {conversation} <<<\n
总结的核心内容如下：\n
"""

map_prompt = PromptTemplate.from_template(format_template)
map_chain = LLMChain(llm = llm, prompt = map_prompt, verbose=False) #, output_key="map_summarization")

# overall summerization template
format_template = """
你是一位认真仔细的会议助手，请根据如下>>> 和 <<<之间的几个分段的会议核心内容，总结一份完整的会议纪要，包括会议主题、会议议程、会议结论以及下一步行动。
要求会议议程不要超过10条，会议结论用一句话简单概括。\n\n
>>> {summerizations} <<<\n
最终总结的会议纪要的如下：\n
"""

reduce_prompt = PromptTemplate.from_template(format_template)
reduce_chain = LLMChain(llm = llm, prompt = reduce_prompt, verbose=False) #, output_key="reduce_summarization")

comb_doc_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="summerizations",
    verbose=False

)

reduce_doc_chain = ReduceDocumentsChain(
    combine_documents_chain=comb_doc_chain,
    collapse_documents_chain=comb_doc_chain,
    token_max=4000,
    verbose=False
)

map_red_chain = MapReduceDocumentsChain(
    llm_chain=map_chain, 
    reduce_documents_chain=reduce_doc_chain,
    document_variable_name="conversation",
    return_intermediate_steps=False,
    verbose=False
)

print(f"###汇总纪要")
print(map_red_chain.run(conversations))

# grounded_prompt = example_prompt.format(summerizations = "\n\n".join(summerizations))
# print(grounded_prompt)
# completion = llm(grounded_prompt)

# print(f"###汇总纪要")
# print(f"{completion}")



NameError: name 'WxYiyan' is not defined